# Soubory

In [ ]:
open("data/leave-me-alone.txt", mode="r", encoding="utf8")

*Vždy* uvádějte `encoding` (pro textové soubory). Python toho [zkousne spostu](https://docs.python.org/3/library/codecs.html#standard-encodings): `utf8` = `utf-8` = `UTF8` = `U8`. Téměř vždy používejte s kontextovým manažerem.

[Módy](https://docs.python.org/3/library/functions.html#open):

- `r` = read = čtení
- `w` = write = zápis (smaže existující data)
- `a` = append = přidávání (zápis za existující obsah)
- `b` = binary = binární režim (jediná možnost vynechat kódování)
- `+` = čtení i zápis
- módy lze kombinovat (`wb+`)

In [ ]:
# vyrobení textového souboru
with open("data/test.txt", mode="w", encoding="utf8") as f:
    f.write("funguje to?\n")
    print("such print, very newline", file=f)

In [ ]:
# přečtení celého souboru najednou
with open("data/test.txt", mode="r", encoding="utf8") as f:
    print(f.read())

In [ ]:
# přečtení po řádcích
with open("data/test.txt", mode="r", encoding="utf8") as f:
    for radka in f:
        print(radka)  # proč dvojí odřádkování?

In [ ]:
# přečtení *najednou* a rozdělení po řádcích
radky = []
with open("data/test.txt", mode="r", encoding="utf8") as f:
    radky = f.readlines()
print(radky)

### Co je pod kapotou?

[I/O streamy](https://docs.python.org/3/library/io.html#module-io):

- `IOBase(object) – close, __enter__, __exit__, __iter__, __next__` a další
- `RawIOBase(IOBase) – read, write` a další
- ... další vrstvy tříd pro bufferování, dekódování apod.


### `StringIO`

= I/O stream – tedy se chová jako soubor – ale nachází se v paměti.

In [ ]:
from io import StringIO

def hloupa_fce(soubor):
    # hloupá funkce pracuje jenom se soubory a nemůžeme to změnit
    soubor.write("hloupá data")

with StringIO() as falesny_soubor:
    hloupa_fce(falesny_soubor)
    print(falesny_soubor.getvalue())

## Systémové proudy

- výstupní: `stdout`
- chybový: `stderr`
- vstupní: `stdin`

In [ ]:
import sys

print("standardní výstup", file=sys.stdout)
print("jejda", file=sys.stderr)

# umožňuje např. shell: `cat data.txt | python3 zpracuj.py`
for radka in sys.stdin:
    print("přijata řádka:", radka)  # dvojí odřádkování ...

## Dej mi vstup!

```python
import fileinput

# zde selže – interaktivní režim
for radka in fileinput.input():
    print(radka)
```

**Použití:**

- `cat data.txt | python3 zpracuj.py`
- `python3 zpracuj.py data.txt`
- `python3 zpracuj.py data.txt data2.txt`
- `python3 zpracuj.py < data.txt`

## Dočasné soubory

In [ ]:
from tempfile import TemporaryFile

# pozor, výchozí mód je `w+b` = binární!
with TemporaryFile(mode="w+", encoding="utf8") as f:
    print("dočasný obsah", file=f)
    
    # kurzor zpět na začátek a přečíst data
    f.seek(0)
    print(f.read())

# Cesty a složky

Dříve kombinace modulů [`os.path`](https://docs.python.org/3/library/os.path.html#module-os.path) a [`glob`](https://docs.python.org/3/library/glob.html), dnes většinově nahrazeno objektovým přístupem – [`pathlib`](https://docs.python.org/3/library/pathlib.html).

Modul `pathlib` se chová "správně" pro Windows / Posix.

- `PurePath` – operace pouze s cestou.
- `Path(PurePath)` – fyzicky sahá na disk.

In [ ]:
from pathlib import Path

# vytvoření objektu cesty
aktualni = Path(".")
print(repr(aktualni))
print(aktualni)  # používá __str__

In [ ]:
# navigace mezi adresáři (spojování cest)
nadrazeny = aktualni / ".." / Path("..") / ".."  # používá __div__
print(nadrazeny)

In [ ]:
# "absolutizace" a vyřazení symlinků
print(nadrazeny.resolve())

In [ ]:
# porovnání (používá __eq__)
print(Path.home() == nadrazeny)  # `home` je absolutní, `nadrazeny` relativní
print(Path.home() == nadrazeny.resolve())  # obě absolutní

In [ ]:
# rozebírání na součástky
verejny_klic = Path.home() / ".ssh" / "id_rsa.pub"
print(verejny_klic.parts)
print(verejny_klic.name)
print(verejny_klic.stem)
print(verejny_klic.suffix)

In [ ]:
# globbing
for notebook in Path(".").glob("**/*.ipynb"):
    print(notebook.parts)

In [ ]:
soubor = Path("./data/test.txt")

# otevírání
with soubor.open(mode="w", encoding="utf8") as f:
    print("Asi brzo umřu..", file=f)
    
# mazání
soubor.unlink()

In [ ]:
# spousta dalších možností ...
dir(Path("."))

# Archivace a komprese

In [ ]:
import shutil

In [ ]:
# zabalení
shutil.make_archive("data/lonely-file", format="zip", base_dir="data/leave-me-alone.txt")

In [ ]:
# rozbalení
shutil.unpack_archive("data/lonely-file.zip", extract_dir="./tmp")

In [ ]:
# úklid ...
Path("data/lonely-file.zip").unlink()
shutil.rmtree("./tmp")

Moduly [`zipfile`](https://docs.python.org/3/library/zipfile.html) a [`tarfile`](https://docs.python.org/3/library/tarfile.html) poskytují širší možnosti, zejména:

- prohlížení obsahu archivu bez rozbalení
- přidávání jednotlivých souborů

=> složitější rozhraní

[`tarfile.open()`](https://docs.python.org/3/library/tarfile.html#tarfile.open):

```
[mód]:[komprese] - "r:*", "w:gz" apod.
```

## Další možnosti

- [zbytek modulu `shutil`](https://docs.python.org/3/library/shutil.html) – Komplexnější operace jako mazání neprázdných složek nebo kopírování souborů skládající se z více elementárních operací.

- [modul `stat`](https://docs.python.org/3/library/stat.html) – Převážně sada konstant pro práci s oprávněními.

# Příklad
Na základě souboru `/etc/passwd` vytvořte soubor `passwd.json`, kam přeneste:
- username
- UID
- common name
- login shell

In [ ]:
from json import dump

# TODO

# Příklad

- Najděte všechny soubory s příponou `.ipynb` v aktuálním adresáři (použijte `pathlib`).
- Projděte je všechny po řádcích (použijte `for` nebo `fileinput`).
- Všechny řádky, které obsahují slovo *"chyba"* vypište na systémový chybový výstup.
- Všechny ostatní řádky vypište do souboru `data/software-bez-chyb.txt`.
- Tento soubor zabalte do archivu `data/software-bez-chyb.tar.gz`.

In [ ]:
import sys
import fileinput
import shutil
from pathlib import Path

# TODO